In [1]:
import numpy as np
import matplotlib.pyplot as plt

Входные данные:

In [4]:
A = np.array([[1, 2]])
b = np.array([2])
c = np.array([3, 4])

### а) Показать, что, если алгоритм центрального пути сходится, то он сходится к решения задачи ЛП

<div style="float: left; width: 30%;">Прямая задача: <br>
max $c^Tx$ <br>
$Ax + x_s = b$ <br>
$x, x_s \geq 0$
</div><div style="float: left; width: 70%;">
Двойственная задача: <br>
min $b^Ty$ <br>
$A^Ty - y_s = c$ <br>
$y, y_s \geq 0$
</div><div class="clear">
<t>На каждом шаге ищем $x, x_s, y, y_s$ такие, что F($x, x_s, y, y_s, \mu$) = $\begin{pmatrix} Ax + x_s - b \\  A^Ty - y_s - c\\ XY_s\textbf1 - \mu\textbf1 \\ X_sY\textbf1 - \mu\textbf1 \end{pmatrix} = \vec0$ 
<br> Уравнения из первых двух строчек системы гарантируют нам, что точки $x, x_s, y, y_s$ являются допустимыми для соответствующих задач ЛП
<br> Из последних двух уравнений получаем, что в пределе ($\mu = 0$) $x^iy_s^i = 0, x_s^jy^j = 0$  $\forall i \in\overline{1,n}, j \in\overline{1,m}  $, из чего при помощи теоремы о комплиментарности делаем вывод, что $x, x_s, y, y_s$ - решения соответствующих задач ЛП

### b) Реализовать алгоритм демпфированного метода Ньютона 
для задачи <br>solve $F(x, 𝑥𝑠, 𝑦, 𝑦𝑠, 𝜇) = \vec0$ <br>$x,x_s,y,y_s>0$

Функция F векторная, метод Ньютона определен для скалярной. Будем решать задачу G($x, x_s, y, y_s, \mu$) = $<F,F>^2$, ее решение, очевидно, является решением исходной. <br>


In [5]:
def G_value(x, x_s, y, y_s, mu):
    r1, r2 = np.dot(A, x) + x_s - b, np.dot(A.traspose(), y) - y_s - c
    v1, v2 = np.array([x[i]*y_s[i] - mu for i in range(len(x))]), np.array([x_s[j]*y[j] - mu for j in range(len(y))])
    return np.dot(r1, r1) + np.dot(r2, r2) + np.dot(v1, v1) + np.dot(v2, v2)

gradG может быть посчитан аналитически <br>
$\frac{\partial G}{\partial x^i} = 2(Ax + x_s -b)A^T[i] + 2(x^iy_s^i - \mu)y_s^i$,   $i \in \overline{1,n}$ <br>
$\frac{\partial G}{\partial x_s^j} = 2(Ax + x_s -b)\textbf1 + 2(x_s^jy^j - \mu)y^j$,   $j \in \overline{1,m}$ <br>
$\frac{\partial G}{\partial y^j} = 2(A^Ty - y_s - c)A[j] + 2(x_s^jy^j - \mu)x_s^j$,   $j \in \overline{1,m}$ <br>
$\frac{\partial G}{\partial y_s^i} = -2(A^Ty - y_s - c)\textbf1 + 2(x^iy_s^i - \mu)x^i$,   $i \in\overline{1,n}$ <br>
Здесь A[i] - i-ая строка матрицы А

In [6]:
def grad_G(x, x_s, y, y_s, mu):
    r1, r2 = np.dot(A, x) + x_s - b, np.dot(A.traspose(), y) - y_s - c
    dot1, dot2 = np.dot(r1, np.ones(len(r1))), np.dot(r2, np.ones(len(r2)))
    grad_x, grad_x_s, grad_y, grad_y_s = np.zeros(len(x)), np.zeros(len(x_s)), np.zeros(len(y)), np.zeros(len(y_s))
    for i in range(len(x)):
        grad_x[i] = 2*np.dot(r1, A.transpose()[i]) + 2*(x[i]*y_s[i] - mu)*y_s[i]
        grad_y_s[i] = -2*dot2 + 2*(x[i]*y_s[i] - mu)*x[i]
    for j in range(len(y)):
        grad_y[j] = 2*np.dot(r2, A[j]) + 2*(x_s[j]*y[j] - mu)*x_s[j]
        grad_x_s[j] = 2*dot1 + 2*(x_s[j]*y[j] - mu)*y[j]
    return grad_x, grad_x_s, grad_y, grad_y_s

В следующей функции определяем, куда нам шагнуть из текущего положения <br>
$x_{n+1} = x_n - \alpha \frac {gradG(x_n)}{|gradG(x_n)|}$ <br>
$\alpha$ выбираем так, чтобы координаты $x_{n+1}$ были неотрицательными <br>
$\alpha_{max} = \frac {G(x_n)}{|gradG(x_n)|}$ <br>
$\alpha = min(\alpha_{max}, min(\frac {x_i}{gradG[i]}, \alpha_{max}))$  $\forall i$ т.ч $gradG[i] > 0$



In [ ]:
def calc_next_x(x, x_s, y, y_s, mu, grad_x, grad_x_s, grad_y, grad_y_s):
    X = np.hstack((x, x_s, y, y_s))
    gradG_X = np.hstack((grad_x, grad_x_s, grad_y, grad_y_s))
    G_X = G_value(x, x_s, y, y_s, mu)
    alpha = G_X/np.linalg.norm(grad_X)
    for i in range(len(X)):
        if gradG_X[i] > 0:
            alpha = min(alpha, X[i]/gradG[i])
    x = 
        
            